<h1> Modeling station activity within a puma. </h1>

I have modeled the flux between regions of citibike stations (pumas). Within a puma however, there can be between 4 and 100+ docking stations for users to acquire or drop off citibikes. I will model the extra- and intra- puma movement.

In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import os
import pickle

In [2]:
pl.rcParams.update({'font.size':22,
                   'figure.figsize':(12,6)})

In [3]:
fn = '/media/mark/TOSHIBA EXT1/Projects/InAPinch/data/citibike/trip_data/2018{:02}-citibike-tripdata.csv'

In [4]:
# Import data.
trip_data = pd.concat([pd.read_csv(fn.format(12)), pd.read_csv(fn.format(11))])

In [5]:
# Remove weird data.
trip_data = trip_data[trip_data['start station id'] != 3239]
trip_data = trip_data[trip_data['start station id'] != 3245]

In [6]:
trip_data.head(4)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,689,2018-12-01 00:00:04.3020,2018-12-01 00:11:33.8460,3359.0,E 68 St & Madison Ave,40.769157,-73.967035,164.0,E 47 St & 2 Ave,40.753231,-73.970325,35033,Subscriber,1989,1
1,204,2018-12-01 00:00:05.5330,2018-12-01 00:03:30.5230,3504.0,E 123 St & Lexington Ave,40.802926,-73.937900,3490.0,E 116 St & 2 Ave,40.796879,-73.937261,20501,Subscriber,1966,1
2,316,2018-12-01 00:00:10.2330,2018-12-01 00:05:27.2030,270.0,Adelphi St & Myrtle Ave,40.693083,-73.971789,243.0,Fulton St & Rockwell Pl,40.688226,-73.979382,18386,Subscriber,1984,1
3,726,2018-12-01 00:00:21.9570,2018-12-01 00:12:28.1830,495.0,W 47 St & 10 Ave,40.762699,-73.993012,3660.0,W 16 St & 8 Ave,40.741022,-74.001385,27616,Subscriber,1983,1


In [7]:
with open('../data/citibike/dock_dict_hist.pickle', 'rb') as f:
    dock_dict = pickle.load(f)
with open('../data/citibike/dock_dict_station.pickle', 'rb') as f:
    dock_dict = {**dock_dict, **pickle.load(f)}
dock_dict.update(dict(zip([3666, 3669, 3441, 3432, 3040, 3719, 3688, 3245], [4101, 4005, 3807, 4005, 4005, 4004, 3808, ])))

In [8]:
# Convert times to datetimes.
trip_data['starttime'] = pd.to_datetime(trip_data['starttime'])
trip_data['stoptime']  = pd.to_datetime(trip_data['stoptime'])

In [9]:
trip_data = trip_data.rename(columns={'start station id': 'startid', 'end station id': 'stopid'}) # Rename columns for convenience.
trip_data = trip_data.dropna(subset={'startid', 'starttime', 'stopid', 'stoptime'})               # Drop rows with important nans.
trip_data.startid = trip_data.startid.astype(int)
trip_data.stopid = trip_data.stopid.astype(int)

In [10]:
trip_data.loc[:, 'start_puma'] = trip_data.startid.map(dock_dict)
trip_data.loc[:, 'stop_puma'] = trip_data.stopid.map(dock_dict)

In [13]:
inds = trip_data.start_puma == trip_data.stop_puma
intra_puma = trip_data[inds]
extra_puma = trip_data[~inds]

In [14]:
extra_puma.groupby('usertype')['usertype'].count()

usertype
Customer        83377
Subscriber    1060499
Name: usertype, dtype: int64

In [15]:
intra_puma.groupby('usertype')['usertype'].count()

usertype
Customer        73211
Subscriber    1059598
Name: usertype, dtype: int64

<h2> Arrival data. </h2>

<h2> Per Station. </h2>

In [16]:
departures = extra_puma[['starttime', 'startid']]
arrivals = extra_puma[['stoptime', 'stopid']]

In [17]:
# Clean departure data.
departures.loc[:, 'puma'] = departures.startid.map(dock_dict)
departures.loc[:, 'dayofyear'] = departures.starttime.dt.dayofyear.copy()
departures.loc[:, 'hour'] = departures.starttime.dt.hour.copy()
departures.loc[:, 'dayofweek'] = departures.starttime.dt.dayofweek.copy()
departures.loc[:, 'weekend'] = departures.starttime.dt.weekday.copy()//5
departures.loc[:, 'outbound'] = -1

/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
# Clean arrival data.
arrivals.loc[:, 'puma'] = arrivals.stopid.map(dock_dict)
arrivals.loc[:, 'dayofyear'] = arrivals.stoptime.dt.dayofyear.copy()
arrivals.loc[:, 'hour'] = arrivals.stoptime.dt.hour.copy()
arrivals.loc[:, 'dayofweek'] = arrivals.stoptime.dt.dayofweek.copy()
arrivals.loc[:, 'weekend'] = arrivals.stoptime.dt.weekday.copy()//5
arrivals.loc[:, 'inbound'] = 1

In [19]:
departures = departures.set_index('starttime')
arrivals = arrivals.set_index('stoptime')

In [20]:
departures = departures.groupby(['startid', 'weekend', 'hour']).outbound.sum().reset_index()
arrivals = arrivals.groupby(['stopid', 'weekend', 'hour']).inbound.sum().reset_index()

In [21]:
# Correct for oversampling.
inds = departures.weekend == 0
departures.loc[inds, 'outbound'] /= 5
departures.loc[~inds, 'outbound'] /= 2

inds = arrivals.weekend == 0
arrivals.loc[inds, 'inbound'] /= 5
arrivals.loc[~inds, 'inbound'] /= 2

In [22]:
# Expand to include all hours.
departures = departures.groupby(['startid', 'weekend'])['hour', 'outbound'].apply(lambda x: x.set_index('hour').reindex(range(0, 24))).reset_index()
departures = departures.interpolate()

arrivals = arrivals.groupby(['stopid', 'weekend'])['hour', 'inbound'].apply(lambda x: x.set_index('hour').reindex(range(0, 24))).reset_index()
arrivals = arrivals.interpolate()

In [76]:
# With the arrivals and departures, compute the activity at each station.
activity = pd.merge(departures, arrivals, how='left', left_on=['startid', 'weekend', 'hour'], right_on=['stopid', 'weekend', 'hour'])
activity.loc[:, 'flux'] = activity.inbound + activity.outbound
activity.head(4)

,startid,weekend,hour,outbound,stopid,inbound,flux
0,72,0,0,-1.8,72,1.2,-0.6
1,72,0,1,-0.8,72,0.6,-0.2
2,72,0,2,-0.2,72,0.5,0.3
3,72,0,3,-0.2,72,0.4,0.2


In [77]:
assert((activity.startid == activity.stopid).all()) # Check that start and stop id's match.

<h2> Per Puma. </h2>

In [59]:
puma_flux = activity[['startid', 'weekend', 'hour', 'flux']]
puma_flux.loc[:, 'puma'] = puma_flux.startid.map(dock_dict)
puma_flux = puma_flux.drop(columns='startid')
puma_flux = puma_flux.groupby(['puma', 'weekend', 'hour'])['flux'].sum().reset_index()
puma_flux.head(4)

/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/mark/anaconda3/envs/inapinch/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,puma,weekend,hour,flux
0,3802,0,0,-1.650000
1,3802,0,1,1.233333
2,3802,0,2,1.066667
3,3802,0,3,-1.700000


<h2> Final activity df. </h2>

In [78]:
activity = activity.rename(columns={'startid':'station_id'})
activity = activity.drop(columns='stopid')
activity.loc[:, 'puma'] = activity.station_id.map(dock_dict)

In [79]:
activity.head(4)

,station_id,weekend,hour,outbound,inbound,flux,puma
0,72,0,0,-1.8,1.2,-0.6,3807
1,72,0,1,-0.8,0.6,-0.2,3807
2,72,0,2,-0.2,0.5,0.3,3807
3,72,0,3,-0.2,0.4,0.2,3807


In [84]:
df = activity.set_index(['weekend', 'hour', 'puma']).join(puma_flux.set_index(['weekend', 'hour', 'puma']), how='left', rsuffix='_puma').reset_index()

In [85]:
df.head(4)

,weekend,hour,puma,station_id,outbound,inbound,flux,flux_puma
0,0,0,3802,3343,-0.6,2.2,1.6,-1.65
1,0,0,3802,3357,-0.6,0.2,-0.4,-1.65
2,0,0,3802,3366,-2.4,1.6,-0.8,-1.65
3,0,0,3802,3383,-3.0,5.0,2.0,-1.65


In [109]:
# max of total number of stations at a puma and 10.
rule = df[['puma', 'station_id']].drop_duplicates().groupby('puma')['station_id'].count().apply(lambda x: max(x, 10))
rule_dict = dict(zip(rule.index.values, rule.values))

In [104]:
df.loc[:, 'share'] = (df.inbound + df.outbound)/df.flux_puma

In [118]:
inds = df.flux_puma.abs() < df.puma.map(rule_dict)
df.loc[inds, 'share'] = 1./df.loc[inds, 'puma'].map(rule_dict)

In [129]:
df = df.set_index(['station_id', 'weekend', 'hour'])

In [130]:
df.to_csv('activity.csv')